In [ ]:
Лабораторна робота №5

з дисципліни "Обробка даних Python"

---

Виконав:

студент групи КН-221Б

Хайлов Данило Євгенович


---


Прийняв:

асистент кафедри

Звєрцев Г.О.

**Мета:** Робота з бібліотекою OpenCV

**Завдання:**
1. Визначити номер варіанта за формулою:
N = ord(X) % 5 + 1, де X – перша велика літера вашого імені в латинській транскрипції.

2. Завантажити дані з файлу kr1.xlsx до датафрейму.
3. Отримати дані до свого варіанту програмно з датафрейму, використовуючи інструменти індексування.
4. Створити функцію, що приймає в якості аргументів дані з датафрейму та повертає зменшене зображення з доданими «круглими окулярами» та зберігає отримане зображення у файл.


*Примітки*
1. При зменшенні зображення помістити обличчя в центрі картинки.
2. Для визначення обличчя та очей використати Haar Cascade з бібліотеки OpenCV.
3. Для малювання використати бібліотеки PIL та/або OpenCV (за бажанням)
4. В якості результату завантажити Jupyter notebook до Google Classroom.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
N = ord('D') % 5 + 1
N

4

In [4]:
import cv2
import pandas as pd
from PIL import Image, ImageDraw

In [5]:
df = pd.read_excel('/content/drive/MyDrive/Data Analysis Python/lab6.xlsx')
df.head()

,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cian,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


In [8]:
my_data = df.iloc[N-1]
my_data

N                             4
file name        longbottom.jpg
image size              600x600
glasses color           magenta
line width                    5
Name: 3, dtype: object

In [9]:
my_data['file name'] = '/content/drive/MyDrive/Data Analysis Python/Images/'+my_data['file name']

<ipython-input-9-494951ae98e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data['file name'] = '/content/drive/MyDrive/Data Analysis Python/Images/'+my_data['file name']


In [10]:
def add_glasses(data):
    color_dict = {
    'red': (0, 0, 255),
    'green': (0, 255, 0),
    'blue': (255, 0, 0),
    'cian': (255, 255, 0),
    'magenta': (255, 0, 255),
    'yellow': (0, 255, 255)
    }

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    img = cv2.imread(data['file name'])

    width, height = map(int, data['image size'].split('x'))
    img = cv2.resize(img, (width, height))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eyes_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
        if len(eyes) >= 2:
            eyes = sorted(eyes, key=lambda x: x[0])
            avg_ew = sum(ew for ex, ey, ew, eh in eyes[:2]) / 2
            for i, (ex, ey, ew, eh) in enumerate(eyes[:2]):
                center = (int(ex + 0.5*ew), int(ey + 0.5*eh))
                radius = int(0.5 * avg_ew)
                color = color_dict[data['glasses color']]
                thickness = data['line width']
                cv2.circle(roi_color, center, radius, color, thickness)
            cv2.line(roi_color, ((eyes[0][0] + eyes[0][2] // 2)+radius, (eyes[0][1] + eyes[0][3] // 2)-3), ((eyes[1][0] + eyes[1][2] // 2)-radius, (eyes[1][1] + eyes[1][3] // 2)+3), color, thickness)
            cv2.line(roi_color, ((eyes[0][0] + eyes[0][2] // 2)-radius, (eyes[0][1] + eyes[0][3] // 2)-2), (eyes[0][0] - eyes[0][2] // 2, eyes[0][1]), color, thickness)
            cv2.line(roi_color, ((eyes[1][0] + eyes[1][2] // 2)+radius, (eyes[1][1] + eyes[1][3] // 2)-2), (eyes[1][0] + 3 * eyes[1][2] // 2, eyes[1][1]), color, thickness)

    cv2.imwrite('output.jpg', img)

In [11]:
add_glasses(my_data)

In [12]:
my_data['glasses color'] = 'green'
my_data['line width'] = 1

<ipython-input-12-6feed4dec6ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data['glasses color'] = 'green'
<ipython-input-12-6feed4dec6ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data['line width'] = 1


In [13]:
add_glasses(my_data)

In [14]:
my_data = df.iloc[3-1]
my_data['file name'] = '/content/drive/MyDrive/Data Analysis Python/Images/'+my_data['file name']

<ipython-input-14-323f4c8300f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_data['file name'] = '/content/drive/MyDrive/Data Analysis Python/Images/'+my_data['file name']


In [15]:
add_glasses(my_data)

**Висновок:** Отримано практичні навички роботи з бібліотекою OpenCV для обробки зображень. Навчилися використовувати різні функції та методи бібліотеки для зчитування, відображення, фільтрації, перетворення, захоплення, збереження та розпізнавання зображень.